# PRÁCTICA 1

Lucía Pérez González, Manuel Ramallo Blanco, Alexandre Lorenzo Martínez

## 1 - Preprocesado

### 1.1 - Eliminación de duplicados

In [45]:
# Abrir datasets
import pandas as pd

df_vino = pd.read_csv("data/train.csv")

# Eliminación de duplicados, ignorado quality
cols = df_vino.columns.drop('quality')
df_vino = df_vino.drop_duplicates(subset=cols)

### 1.2 - Binarización de la calidad

In [46]:
def clasificar_vino(valor): 
    if valor < 7: 
        return "baja calidad" 
    elif valor >= 7: 
        return "alta calidad" 
df_vino['calidad'] = df_vino['quality'].apply(clasificar_vino) 
df_vino = df_vino.drop(columns=['quality'])

### 1.3 - Gestión de valores atípicos

In [ ]:
# Deteccion de valores atipicos

def detectar_atipicos(df_train, df_val, cols_diana):
    for col in cols_diana:

        # Calculamos IQR y límites
        Q1 = df_train[col].quantile(0.25)
        Q3 = df_train[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 6 * IQR
        upper = Q3 + 6 * IQR
        
        # Declaramos condición de atípico y registramos sus posiciones
        cond_iqr = (df_train[col] < lower) | (df_train[col] > upper)    #OLLO! Nico recomendara ver os casos 1 a 1 
        cond_neg = df_train[col] < 0
        cond_atipico = cond_iqr | cond_neg


        # Incrementamos el contador de atípicos por fila en ambos conjuntos
        df_train.loc[cond_atipico, col] = pd.NA
        df_val.loc[cond_atipico, col] = pd.NA


    return df_train, df_val


Número de filas con 0, 1, 2, ... atípicos (IQR + negativos):
num atipicos
0    1829
1    1142
2     516
3     184
4      50
5       8
6       3
Name: count, dtype: int64


#### 1.3.11 - Tratamiento de datos atípicos

In [ ]:
def eliminar_o_imputacion(df_train, df_val, cols_diana, max_atipicos=3, umbral_col=20,  target='calidad'):
    # Eliminar filas con 4 o más valores atípicos
    # Contamos el número de valores atípicos por fila (NA) y filtramos 
    df_train = df_train[df_train.isnull().sum(axis=1) < max_atipicos]

    # Analisis de valores faltantes por columna
    for col in cols_diana:
        # Contamos el numero de valores faltantes (NA) en la columna(
        num_faltantes = df_train[col].isna().sum()
        if num_faltantes/len(df_train) > umbral_col:
            df_train = df_train.drop(columns=[col])
            df_val = df_val.drop(columns=[col])

    # Imputacion de valores: mediana para cada nulo de cada columna
    for col in cols_diana:
        med = df_train[col].median()
        df_train[col] = df_train[col].fillna(med)
        df_val[col] = df_val[col].fillna(med)

    return df_train, df_val


#### Normalización y selección de características

In [ ]:
# Normalización de los datos
from sklearn.preprocessing import StandardScaler

def estandarizar_train_test(train_df, test_df, target):
    columnas = train_df.drop(columns=[target]).select_dtypes(include="number").columns

    scaler = StandardScaler()


    train_df[columnas] = scaler.fit_transform(train_df[columnas])
    test_df[columnas] = scaler.transform(test_df[columnas])

    return train_df, test_df


In [2]:
# Selección de características con SelectKBest
from sklearn.feature_selection import SelectKBest, f_regression

def seleccion_caracteristicas(df, k, target='calidad'):
    X = df.drop(columns=[target])
    y = df[target]

    selector = SelectKBest(score_func=f_regression, k=k)
    selector.fit(X, y)

    selected_features = X.columns[selector.get_support()]

    df_fs = df[selected_features.tolist() + [target]]
    return df_fs


#### Función que empaqueta el preprocesado

In [ ]:
def preprocesado(train_df_pre, valid_df_pre, target='calidad'):
    df_train = train_df_pre.copy()
    df_val = valid_df_pre.copy()
    cols_diana = [c for c in train_df_pre.columns if c not in [target]]
    
    # Detección de atípicos en train y valid
    df_train, df_val = detectar_atipicos(df_train, df_val, cols_diana)

    # Tratamiento de atípicos: eliminación o imputación
    df_train, df_val = eliminar_o_imputacion(df_train, df_val, cols_diana)

    # Normalización de los datos
    df_train, df_val = estandarizar_train_test(df_train, df_val, target)

    # Selección de características con SelectKBest
    df_train = seleccion_caracteristicas(df_train, k=10, target=target)
    df_val = df_val[df_train.columns]
    return df_train, df_val 


## Entrenamiento del modelo de predicción

In [3]:
# Modelo de entrenamiento
import numpy as np
import pandas as pd

def nested_cv(df, target='calidad'):
    """
    Validación cruzada anidada:
    - exterior: 3 iteraciones (6 pedazos, 4 train, 2 val)
    - interior: 4 pedazos (3 train, 1 val)
    
    Devuelve: diccionario con información de folds
    """
    
    # Mezclar el dataset para aleatoriedad
    df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
    n = len(df_shuffled)
    
    # Dividir en 6 pedazos iguales (outer)
    outer_splits = np.array_split(df_shuffled, 6)
    
    results = []
    
    # Outer CV
    for outer_iter in range(3):
        # Elegimos 4 pedazos para train, 2 para val
        outer_train_idx = [(outer_iter + i) % 6 for i in range(4)]
        outer_valid_idx = [(outer_iter + 4 + i) % 6 for i in range(2)]
        
        train_outer = pd.concat([outer_splits[i] for i in outer_train_idx])
        valid_outer = pd.concat([outer_splits[i] for i in outer_valid_idx])
        
        # Dividir train_outer en 4 pedazos para inner CV
        inner_splits = np.array_split(train_outer.sample(frac=1, random_state=outer_iter), 4)
        
        inner_results = []
        
        # Inner CV
        for inner_iter in range(4):
            inner_valid_pre = inner_splits[inner_iter]
            inner_train_pre = pd.concat([s for j, s in enumerate(inner_splits) if j != inner_iter])
            
            # Preprocesar
            df_train, df_val = preprocesado(inner_train_pre, inner_valid_pre, target)
            
            # --- Aquí va el entrenamiento de los modelos ---

            # Guardar info del cada ejecución del fold interno
            inner_results.append({
                "inner_iter": inner_iter,
                "train_rows": len(df_train),
                "valid_rows": len(df_val),
                # "score": score
            })
        
        # Preprocesar outer validation usando train outer
        train_outer_proc, valid_outer_proc = preprocesado(train_outer, valid_outer, target)
        
        # aqui entreno en train_outer_proc el modelo con mejor puntuación del la VC interna y evalúo en valid_outer_proc
        

        results.append({
            "outer_iter": outer_iter,
            "train_rows_outer": len(train_outer_proc),
            "valid_rows_outer": len(valid_outer_proc),
            "inner_folds": inner_results,
            # "outer_score": outer_score
        })

    return results
